In [1]:
#Importing libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#Set the url to scrape
url = 'https://www.techpowerup.com/gpu-specs/?released=2022&directx=12%20Ultimate&igp=No&sort=name'
#Make a connection to the url
page = requests.get(url)
#If connection is successful, scrape the contents of the url
if page.status_code == 200:
    soup = BeautifulSoup(page.content, 'html.parser')
    print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   GPU Database | TechPowerUp
  </title>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="https://tpucdn.com" rel="preconnect"/>
  <meta content="TechPowerUp" name="application-name"/>
  <meta content="Graphics card and GPU database with specifications for products launched in recent years. Includes clocks, photos, and technical details." name="description"/>
  <link as="style" crossorigin="" href="https://tpucdn.com/css/style-v1701979023067.css" rel="preload stylesheet"/>
  <link as="font" crossorigin="" href="https://tpucdn.com/css/fonts/tpu-body-regular-v1701979023067.woff2" rel="preload" type="font/woff2"/>
  <link as="font" crossorigin="" href="https://tpucdn.com/css/fonts/tpu-body-bold-v1701979023067.woff2" rel="preload" type="font/woff2"/>
  <link as="font" crossorigin="" href="https://tpucdn.com/css/fonts/opensans-condbold-v1701979023067.woff2" rel="preload" type="f

In [3]:
#Create varaible table to narrow down the elements of the HTML to scrape
table = soup.find_all('table', class_ = 'processors')

In [4]:
#Create lists for each piece of data that is collected
name = []
chip = []
date = []
bus = []
vram = []
ddr = []
bit = []
gclock = []
mclock = []
shaders = []
tmus = []
rops = []

#Extracts all the td tags from the table variable. Further narrows scraping.
for i in table:
    tag_td = i.find_all('td')

#Create a counter variable for the for loop below that appends data to a specific list based on the count.
count = 0
#For loop that looks through every tr tag inside a td tag, get its text, then moves to the next td tag.
for tag_tr in tag_td:
    #Variable element used to easily modify selected text, if needed.
    element = tag_tr.get_text()
    #If & elif statements check which category to assign data to.
    #Count 0 assigns data to name list.
    if count == 0:
        #Removes \n from text, then appends text to name list.
        element = element.replace('\n', '')
        name.append(element)
        count += 1 #Move on to next section
    #Count 1 assigns data to chip list.
    elif count == 1:
        #Removes \n from text, then appends text to name list.
        element = element.replace('\n', '')
        chip.append(element)
        count += 1 #Move on to next section
    elif count == 2:
    #Count 2 assigns data to the date list.
        #Appends text to the date list.
        date.append(element)
        count += 1 #Move on to next section
    #Count 3 assigns data to the bus list.
    elif count == 3:
        #Remove x to extract the integer easier.
        element = element.replace('x', '')
        #Split the text into a list.
        element = element.split(' ')
        #Append integer from the split list to the bus list.
        bus.append(int(element[2]))
        count += 1 #Move on to next section
    #Count 4 modifies the memory column into 3 separate columns.
    elif count == 4:
        #Split the text into a list called memory.
        memory = element.split(', ')
        #Index 0 is the VRAM. Remove GB and extract only the integer. Append to vram list.
        vram.append(int(memory[0].replace(' GB', '')))
        #Index 1 is the memory type. Append to ddr list.
        ddr.append(memory[1])
        #Index 2 is the bus width. Remove bit and extract only the integer. Append to bit list.
        bit.append(int(memory[2].replace(' bit', '')))
        count += 1 #Move on to next section
    #Count 5 assigns data to the gclock list.
    elif count == 5:
        #Remove MHz to only extract the number.
        element = element.replace(' MHz', '')
        #Append nummber into gclock list as an integer.
        gclock.append(int(element))
        count += 1 #Move on to next section
    #Count 6 assigns data to the mclock list.
    elif count == 6:
        #Remove MHz to only extract the number.
        element = element.replace(' MHz', '')
        #Append nummber into mclock list as an integer.
        mclock.append(int(element))
        count += 1 #Move on to next section
    #Count 7 splits the Shaders / TMUs / ROPs column into their own column.
    #Allows numbers to be compared more easily.
    elif count == 7:
        #Splits the numbers into a list called str.
        srt = element.split(' / ')
        #Index 0 is the shaders. Append to shaders list as an integer.
        shaders.append(int(srt[0]))
        #Index 1 is the TMUs. Append to tmus list as an integer.
        tmus.append(int(srt[1]))
        #Index 2 is the ROPs. Append to rops list as an integer.
        rops.append(int(srt[2]))
        count = 0 #This is the last column, start from name category.

In [5]:
#Create a dictionary to be used for the dataframe.
#Brings all the data into one variable, dict_.
dict_ = {
    'Name' : name,
    'Chip' : chip,
    'Date Released' : date,
    'PCIe Bus Lanes (Gen 4)' : bus,
    'VRAM (in GB)' : vram,
    'Memory Type' : ddr,
    'Bus Width (in Bit)' : bit,
    'GPU Clock (in MHz)' : gclock,
    'Memory Clock (in MHz)' : mclock,
    'Shaders' : shaders,
    'TMUs' : tmus,
    'ROPs' : rops
}

In [11]:
#Create the dataframe, df, using the dict_ dictionary.
df = pd.DataFrame(dict_)
#Removing Tesla T40 24 GB since it is PCIe Gen 3.
df = df.drop(index=81)
df

,Name,Chip,Date Released,PCIe Bus Lanes (Gen 4),VRAM (in GB),Memory Type,Bus Width (in Bit),GPU Clock (in MHz),Memory Clock (in MHz),Shaders,TMUs,ROPs
0,Radeon PRO W6300,Navi 24,"Jan 19th, 2022",4,2,GDDR6,32,1512,2000,768,48,32
1,Radeon PRO W6400,Navi 24,"Jan 19th, 2022",4,4,GDDR6,64,2039,2000,768,48,32
2,Radeon Pro W6300M,Navi 24,"Jan 19th, 2022",4,2,GDDR6,32,1512,2000,768,48,32
3,Radeon Pro W6500M,Navi 24,Never Released,4,4,GDDR6,64,1512,2000,1024,64,32
4,Radeon RX 6300,Navi 24,Never Released,4,2,GDDR6,32,1000,2000,768,48,32
...,...,...,...,...,...,...,...,...,...,...,...,...
76,RTX A500 Embedded,GA107S,"Mar 30th, 2022",8,2,GDDR6,64,435,1750,2048,64,32
77,RTX A500 Mobile,GA107S,"Mar 22nd, 2022",8,4,GDDR6,64,832,1500,2048,64,32
78,RTX A5500,GA102,"Mar 22nd, 2022",16,24,GDDR6,384,1080,2000,10240,320,96
79,RTX A5500 Max-Q,GA103,"Mar 22nd, 2022",16,16,GDDR6,256,585,1750,7424,232,96


In [7]:
#Gather general statistics from applicable columns
df.describe()

,PCIe Bus Lanes (Gen 4),VRAM (in GB),Bus Width (in Bit),GPU Clock (in MHz),Memory Clock (in MHz),Shaders,TMUs,ROPs
count,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000
mean,11.853659,11.707317,193.560976,1403.329268,1831.500000,4714.146341,181.658537,79.414634
std,4.637799,9.712519,111.408945,548.282145,346.543185,4517.198310,142.152693,52.448670
min,4.000000,2.000000,32.000000,435.000000,1093.000000,768.000000,32.000000,16.000000
25%,8.000000,4.000000,96.000000,918.750000,1750.000000,1856.000000,64.000000,32.000000
50%,16.000000,8.000000,192.000000,1500.000000,1812.500000,2816.000000,128.000000,64.000000
75%,16.000000,16.000000,256.000000,1907.250000,2000.000000,5888.000000,252.000000,96.000000
max,16.000000,48.000000,384.000000,2321.000000,2500.000000,18176.000000,568.000000,192.000000
